In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import shapiro, levene, ttest_ind, mannwhitneyu
import numpy as np


# from IPython.display import display
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 50)
# pd.set_option('display.expand_frame_repr', True)
# import warnings
# warnings.filterwarnings('ignore')



In [50]:
## Parameters: when running manually
# study_name = "vox"
# experiments = "crosspropxv4s30"
# runs = "1"
# generations = 19
# voxel_types "withbone"
# final_gen = 19
# ### path out of docker
# out_path = "/home/ripper8/projects/working_data"


In [51]:
voxel_types_list = voxel_types.split(",")

analysis_path = f'{out_path}/{study_name}/analysis'

def parse_csv_list(s, cast=int):
    return [cast(x.strip()) for x in str(s).split(",") if x.strip()]

runs = parse_csv_list(runs, int)      # "1,2" -> [1, 2], "3" -> [3]
experiments = parse_csv_list(experiments, str)

gens_robots_outer = pd.read_csv(f'{analysis_path}/gens_robots_outer.csv')
gens_robots_inner = pd.read_csv(f'{analysis_path}/gens_robots_inner.csv')


In [ ]:
### Metrics progression ###

# metrics =  [
#     "fitness",
#     "uniqueness",
#     "novelty",
#     "age",
#     "genome_size",
#     "displacement",
#     "num_voxels",
#     "bone_count",
#     "bone_prop",
#     "fat_count",
#     "fat_prop",
#     "fat2_count",
#     "fat2_prop",
#     "phase_muscle_count",
#     "phase_muscle_prop",
#     "offphase_muscle_count",
#     "offphase_muscle_prop",
# ]
# metrics_labels =  [
#     "fitness",
#     "uniqueness",
#     "novelty",
#     "age",
#     "genome_size",
#     "displacement",
#     "num_voxels",
#     "bone_count",
#     "bone_prop",
#     "fat_count",
#     "fat_prop",
#     "phase_muscle_count",
#     "phase_muscle_prop",
#     "offphase_muscle_count",
#     "offphase_muscle_prop",
# ]

metrics = [

    # -------------------------
    # Genotypic
    # -------------------------
    "genome_size",

    # -------------------------
    # Behavioral
    # -------------------------
    "displacement",

    # -------------------------
    # Phenotypic – counts & proportions
    # -------------------------
    "num_voxels",

    "bone_count",
    "bone_prop",

    "fat_count",
    "fat_prop",

    "fat2_count",
    "fat2_prop",

    "phase_muscle_count",
    "phase_muscle_prop",

    "offphase_muscle_count",
    "offphase_muscle_prop",

    "phase_ratio",  # optional (phase / total muscle)

    # -------------------------
    # Contact / Ground interaction
    # -------------------------
    "contact_area",
    "contact_prop",
    "contact_muscle_prop",
    "contact_passive_prop",
    "contact_bone_prop",

    # -------------------------
    # Vertical layering
    # -------------------------
    "mean_z_bone",
    "mean_z_fat",
    "mean_z_fat2",
    "mean_z_phase_muscle",
    "mean_z_offphase_muscle",
    "mean_z_passive",
    "mean_z_muscle",
    "layering_muscle_minus_passive",

    # -------------------------
    # Shape / Bounding box
    # -------------------------
    "extent_x",
    "extent_y",
    "extent_z",
    "bbox_fill",
    "elongation_xy",
    "flatness",
    "minmax_extent_ratio",  # optional

    # -------------------------
    # Surface / Connectivity
    # -------------------------
    "surface_voxels",
    "surface_ratio",
    "avg_connectivity",  # optional if you add it

    # -------------------------
    # Symmetry
    # -------------------------
    "mirror_x_diff",
    "mirror_y_diff",

    # -------------------------
    # Muscle organization
    # -------------------------
    "muscle_components",
    "largest_muscle_component_prop",

    # -------------------------
    # Relative metrics
    # -------------------------
    "uniqueness",
    "novelty",
    "novelty_weighted",
    "dominated_disp_nov",
    "fitness",
    "age",
]

metrics_labels = [

    # Genotypic
    "Genome Size",

    # Behavioral
    "Displacement (+X)",

    # Basic phenotype
    "Number of Voxels",

    "Bone Count",
    "Bone Proportion",

    "Fat Count",
    "Fat Proportion",

    "Fat2 Count",
    "Fat2 Proportion",

    "Phase Muscle Count",
    "Phase Muscle Proportion",

    "Off-Phase Muscle Count",
    "Off-Phase Muscle Proportion",

    "Phase Ratio (Phase / Total Muscle)",

    # Contact
    "Ground Contact Area",
    "Ground Contact Proportion",
    "Ground Muscle Contact Proportion",
    "Ground Passive Contact Proportion",
    "Ground Bone Contact Proportion",

    # Layering
    "Mean Z Bone",
    "Mean Z Fat",
    "Mean Z Fat2",
    "Mean Z Phase Muscle",
    "Mean Z Off-Phase Muscle",
    "Mean Z Passive",
    "Mean Z Muscle",
    "Layering (Muscle - Passive)",

    # Shape
    "Extent X",
    "Extent Y",
    "Extent Z",
    "Bounding Box Fill Ratio",
    "Elongation XY",
    "Flatness (Z / max(X,Y))",
    "Min/Max Extent Ratio",

    # Surface
    "Surface Voxels",
    "Surface Ratio",
    "Average Connectivity",

    # Symmetry
    "Mirror Asymmetry X",
    "Mirror Asymmetry Y",

    # Muscle organization
    "Muscle Connected Components",
    "Largest Muscle Component Proportion",

    # Relative
    "Uniqueness (Pop Distance)",
    "Novelty (Archive kNN)",
    "Novelty-Weighted Fitness",
    "Pareto Dominance (Disp + Nov)",
    "Fitness",
    "Age",
]

# options: "fitness" (only fitness), "all" (all metrics), "none" (no max curves)
SHOW_MAX_MODE = "all"
clrs = ['#ADD8E6', '#ADD8E6', '#F6B970', '#F6B970']

def cols(prefix):
    """Return (median, q1, q3) column names for a given prefix."""
    return f'{prefix}_median', f'{prefix}_q25', f'{prefix}_q75'

def choose_test(a, b):
    """Return (label, p, effect_label, effect_value)."""
    a = pd.Series(a).dropna()
    b = pd.Series(b).dropna()
    if len(a) < 2 or len(b) < 2:
        return ("n<2", np.nan, "", np.nan)

    # Normality (only meaningful for n>=3; else force nonparametric)
    if len(a) >= 3 and len(b) >= 3:
        p_norm_a = shapiro(a).pvalue
        p_norm_b = shapiro(b).pvalue
        normal = (p_norm_a > 0.05) and (p_norm_b > 0.05)
    else:
        normal = False

    if normal:
        # Homoskedasticity
        p_lev = levene(a, b).pvalue
        equal_var = p_lev > 0.05
        # Student (equal_var=True) or Welch (equal_var=False)
        t_res = ttest_ind(a, b, equal_var=equal_var)
        p = t_res.pvalue
        # Hedges' g
        n1, n2 = len(a), len(b)
        m1, m2 = a.mean(), b.mean()
        s1, s2 = a.std(ddof=1), b.std(ddof=1)
        sp = np.sqrt(((n1-1)*s1**2 + (n2-1)*s2**2) / (n1+n2-2)) if (n1+n2-2) > 0 else np.nan
        d = (m1 - m2) / sp if sp and sp > 0 else np.nan
        J = 1 - (3 / (4*(n1+n2) - 9)) if (n1+n2) > 2 else 1.0
        g = d * J
        label = "Student t" if equal_var else "Welch t"
        return (label, p, "Hedges g", g)
    else:
        # Mann–Whitney U (two-sided)
        u_res = mannwhitneyu(a, b, alternative='two-sided')
        p = u_res.pvalue
        n1, n2 = len(a), len(b)
        # Cliff's delta from U
        delta = (2*u_res.statistic/(n1*n2)) - 1
        return ("Mann–Whitney U", p, "Cliff's Δ", delta)

for idm, metric in enumerate(metrics):

    font = {'font.size': 20}
    plt.rcParams.update(font)
    fig, ax = plt.subplots()

    plt.xlabel('')
    plt.ylabel(f'{metrics_labels[idm]}')
    ax.grid(False)
    for side in ('top','right','bottom','left'):
        ax.spines[side].set_color('grey')
        ax.spines[side].set_linewidth(0.5)
    ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=True)
    ax.tick_params(axis='y', which='both', left=False, right=False, labelleft=True)

    # use MEAN aggregates for every metric (outer curves)
    mean_prefix = f'{metric}_mean'
    mean_med_col, mean_q1_col, mean_q3_col = cols(mean_prefix)

    # decide whether to also show MAX aggregates for this metric
    show_max_this_metric = (
        (SHOW_MAX_MODE == "all") or
        (SHOW_MAX_MODE == "fitness" and metric == "fitness")
    )

    # prepare MAX columns if enabled
    if show_max_this_metric:
        max_prefix = f'{metric}_max'
        max_med_col, max_q1_col, max_q3_col = cols(max_prefix)

    # plot curves
    for idx_experiment, exp in enumerate(experiments):
        metric_exp = gens_robots_outer[gens_robots_outer['experiment'] == exp]

        ax.plot(
            metric_exp['generation'], metric_exp[mean_med_col],
            label=f'{exp} (mean)', c=clrs[idx_experiment], linewidth=2
        )
        ax.fill_between(
            metric_exp['generation'],
            metric_exp[mean_q1_col],
            metric_exp[mean_q3_col],
            alpha=0.35, facecolor=clrs[idx_experiment]
        )
        ax.plot(metric_exp['generation'], metric_exp[mean_q1_col],
                linestyle='-', color=clrs[idx_experiment], alpha=0.5, linewidth=1.5)
        ax.plot(metric_exp['generation'], metric_exp[mean_q3_col],
                linestyle='-', color=clrs[idx_experiment], alpha=0.5, linewidth=1.5)

        if show_max_this_metric:
            ax.plot(
                metric_exp['generation'], metric_exp[max_med_col],
                label=f'{exp} (max)', c=clrs[idx_experiment], linewidth=1, linestyle='--'
            )
            ax.fill_between(
                metric_exp['generation'],
                metric_exp[max_q1_col],
                metric_exp[max_q3_col],
                alpha=0.2, facecolor=clrs[idx_experiment]
            )

    # ----- statistical test at final generation using INNER df -----
    if len(experiments) >= 2:
        exp_a, exp_b = experiments[0], experiments[1]

        # compare per-run inner MEANS at final generation
        gA = gens_robots_inner[(gens_robots_inner['experiment'] == exp_a) &
                               (gens_robots_inner['generation'] == final_gen)][f'{metric}_mean']
        gB = gens_robots_inner[(gens_robots_inner['experiment'] == exp_b) &
                               (gens_robots_inner['generation'] == final_gen)][f'{metric}_mean']

        test_label, pval, eff_label, eff_val = choose_test(gA, gB)
        annot_lines = [f"{metric} mean: {test_label} p={pval:.3g}"]

        if show_max_this_metric:
            # compare per-run inner MAX at final generation
            gA_max = gens_robots_inner[(gens_robots_inner['experiment'] == exp_a) &
                                       (gens_robots_inner['generation'] == final_gen)][f'{metric}_max']
            gB_max = gens_robots_inner[(gens_robots_inner['experiment'] == exp_b) &
                                       (gens_robots_inner['generation'] == final_gen)][f'{metric}_max']

            test_label2, pval2, eff_label2, eff_val2 = choose_test(gA_max, gB_max)
            annot_lines.append(f"{metric} max: {test_label2} p={pval2:.3g}")

        ax.text(
            0.02, 0.98, "\n".join(annot_lines),
            transform=ax.transAxes, ha="left", va="top",
            fontsize=8.5,
            linespacing=0.85,
            bbox=dict(
                boxstyle="round,pad=0.12",
                fc="white", ec="gray", lw=0.6, alpha=0.8
            ),
            zorder=10, clip_on=False
        )

    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1),
              fancybox=True, shadow=True, ncol=5, fontsize=10)

    fig.savefig(f'{analysis_path}/progression_{metric}.png', dpi=300, bbox_inches='tight')
    plt.show()
    plt.close(fig)

